In [7]:
!nvidia-smi
!which python | grep DYY

Sun Sep 22 08:16:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000004:04:00.0 Off |                    0 |
| N/A   41C    P0              40W / 300W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
import torch
torch.manual_seed(3407)
torch.backends.cudnn.deterministic = True

# Model

In [9]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class Res(nn.Module):
#     def __init__(self, channel, kernel_size=3,x_size = 28) -> None:
#         super().__init__()
#         self.channel = channel
#         self.conv1=nn.Sequential(
#             nn.Conv2d(channel,channel,(kernel_size,kernel_size),padding=(kernel_size-1)//2),
#             nn.BatchNorm2d(channel),
#         )
#         self.conv2= nn.Sequential(
#             nn.ReLU(),
#             nn.Conv2d(channel,channel,(kernel_size,kernel_size),padding=(kernel_size-1)//2),
#         )
#         self.t_net = nn.Linear(256,self.channel*x_size*x_size)
#     def forward(self,x,t):
#         res = x.clone()
#         x = self.conv1(x)
#         x = x + self.t_net(t).reshape(x.shape)
#         x = self.conv2(x) + res
#         return x

# class SinousEmbedding(nn.Module):
#     def __init__(self, dim) -> None:
#         super().__init__()
#         assert dim%2==0,NotImplementedError()
#         self.angles = (10000**(-2/dim))**torch.arange(1,dim//2+1,1).cuda()
#         self.angles.requires_grad_(False)
#     def forward(self,x):
#         angles = torch.einsum('m,i->im',self.angles,x)
#         return torch.cat((torch.sin(angles),torch.cos(angles)),dim=1)

# class Attention(nn.Module):
#     def __init__(self, channel,hidden_size=512) -> None:
#         super().__init__()
#         self.hidden_size = hidden_size
#         self.q_proj = nn.Linear(channel,hidden_size)
#         self.k_proj = nn.Linear(channel,hidden_size)
#         self.v_proj = nn.Linear(channel,hidden_size)
#         self.out_proj = nn.Linear(hidden_size,channel)
#     def forward(self,x):
#         res = x.clone()
#         batch,channel = x.shape[:2]
#         seq_len = x.shape[-1]
#         x = x.reshape(batch,channel,seq_len*seq_len).transpose(1,2)
#         v = self.v_proj(x)
#         q = self.q_proj(x)
#         k = self.k_proj(x)
#         att_sc = torch.einsum('bic,bjc->bij',q,k)*((self.hidden_size)**-0.5)
#         att_sc = torch.softmax(att_sc,dim=-1)
#         att_out = torch.einsum('bij,bjc->bic',att_sc,v)
#         ans = self.out_proj(att_out).transpose(1,2).reshape(batch,channel,seq_len,seq_len)
#         return ans+res


# class ResBlockWithAttention(nn.Module):
#     def __init__(self, in_channel, out_channel,x_size,with_attention=True,kernel_size=3) -> None:
#         super().__init__()
#         self.conv=nn.Sequential(
#             nn.Conv2d(in_channel,out_channel,(kernel_size,kernel_size),padding=(kernel_size-1)//2),
#             nn.BatchNorm2d(out_channel),
#             nn.ReLU(),
#         )
#         self.reses = nn.ModuleList(
#             [Res(out_channel,kernel_size,x_size) for _ in range(4)]
#         )
#         if with_attention:
#             self.attentions = nn.ModuleList(
#                 [Attention(out_channel) for _ in range(4)]
#             )
        
#     def forward(self,x,t):
#         x = self.conv(x)
#         for i,ly in enumerate(self.reses):
#             x = ly(x,t)
#             if hasattr(self,'attentions'):
#                 x = self.attentions[i](x)
#         return x


# class DDPM(nn.Module):
#     def __init__(self, *args, **kwargs) -> None:
#         super().__init__(*args, **kwargs)
#         self.in_size = 28 * 28
#         self.t_embedding = SinousEmbedding(dim=256)
#         self.up= nn.ModuleList([
#             ResBlockWithAttention(1,64,x_size=28,with_attention=False), # 28 28
#             nn.MaxPool2d(kernel_size=(2,2)), # 14 14
#             ResBlockWithAttention(64,128,x_size=14), # 14 14
#             nn.MaxPool2d(kernel_size=(2,2)), # 7 7
#             ResBlockWithAttention(128,256,x_size=7), # 7 7
#         ])
#         self.middle = nn.ModuleList([
#             nn.Conv2d(256,256,kernel_size=(5,5),padding=2),
#             nn.ReLU(),
#             Attention(256)
#         ])
#         self.down= nn.ModuleList([
#             ResBlockWithAttention(512,128,x_size=7), # 7 7
#             nn.Upsample(scale_factor=2),
#             ResBlockWithAttention(256,64,x_size=14),
#             nn.Upsample(scale_factor=2),
#             ResBlockWithAttention(128,1,x_size=28,with_attention=False),       
#         ])

#     def forward(self,x,t):
#         x = x.reshape(-1,1,28,28)
#         ttensor = self.t_embedding(t)
#         batch = x.shape[0]
#         ups = []
#         for i,ly in enumerate(self.up):
#             if isinstance(ly,ResBlockWithAttention):
#                 x = ly(x,ttensor)
#                 cl = x.clone()
#                 ups.append(cl)
#             else:
#                 x = ly(x)
#         for ly in self.middle:
#             x = ly(x)
#         for ly in self.down:
#             if isinstance(ly,ResBlockWithAttention):
#                 old = ups.pop()
#                 x = ly(torch.cat((x,old),dim=1),ttensor)
#             else:
#                 x = ly(x)
#         x = x.reshape(batch,-1)
#         return x

In [10]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class SinousEmbedding(nn.Module):
#     def __init__(self, dim) -> None:
#         super().__init__()
#         assert dim%2==0,NotImplementedError()
#         self.angles = (1000.**(-2/dim))**torch.arange(1,dim//2+1,1,dtype=torch.float).cuda()
#         self.angles.requires_grad_(False)
#     def forward(self,x):
#         angles = torch.einsum('m,i->im',self.angles,x.float())
#         return torch.cat((torch.sin(angles),torch.cos(angles)),dim=1)

# class DDPM(nn.Module):
#     def __init__(self, *args, **kwargs) -> None:
#         super().__init__(*args, **kwargs)
#         self.in_size = 28 * 28
#         self.t_embedding_dim = 256
#         self.t_embedding = SinousEmbedding(dim=self.t_embedding_dim)
#         self.up = nn.ModuleList([
#             nn.Sequential(
#                 nn.Linear(784+self.t_embedding_dim,64),
#                 nn.ReLU(),
#             ),
#             nn.Sequential(
#                 nn.Linear(64,32),
#                 nn.ReLU(),
#             ),
#             # nn.Sequential(
#             #     nn.Linear(256,256),
#             #     # nn.LeakyReLU(0.1),
#             # ),
#         ])
#         self.middle = nn.ModuleList([
#             nn.Linear(32,32),
#             # nn.LeakyReLU(0.1),
#         ])
#         self.down= nn.ModuleList([
#             nn.Sequential(
#                 nn.Linear(32,32),
#                 nn.ReLU(),
#             ),
#             # nn.Sequential(
#             #     nn.Linear(256,256),
#             #     # nn.LeakyReLU(0.1),
#             # ),
#             nn.Sequential(
#                 nn.Linear(32,64),
#                 nn.ReLU(),
#             ),
#         ])
#         self.end_mlp = nn.Linear(64,784)
#         self.apply_init()

#     def apply_init(self):
#         for m in self.modules():
#             if isinstance(m, nn.Linear):
#                 nn.init.xavier_normal_(m.weight)
#                 nn.init.constant_(m.bias, 0)

#     def forward(self,x,t):
#         x = x.reshape(-1,784)
#         ttensor = self.t_embedding(t) # [batch, 256]
#         batch = x.shape[0]
#         xc = x.clone()
#         ups = []
#         x = torch.cat((x,ttensor),dim=-1)
#         for ly in self.up:
#             x = ly(x)
#             ups.append(x.clone())
#         for ly in self.middle:
#             x = ly(x)
#         for ly in self.down:
#             x = ly(x) + ups.pop()

#         x = self.end_mlp(x)
#         x = (x + xc)
#         return x

In [11]:
def print_range(x):
    print('min:',x.min().item(),'max:',x.max().item(),'std:',x.std().item())

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SinousEmbedding(nn.Module):
    def __init__(self, dim) -> None:
        super().__init__()
        assert dim%2==0,NotImplementedError()
        self.angles = (10000.**(-2/dim))**torch.arange(1,dim//2+1,1,dtype=torch.float).cuda()
        self.angles.requires_grad_(False)
    def forward(self,x):
        angles = torch.einsum('m,i->im',self.angles,x.float())
        return torch.cat((torch.sin(angles),torch.cos(angles)),dim=1)
    
class ResidualBlock(nn.Module):

    def __init__(self,channels=128,kernel_size=3,t_dim=64) -> None:
        super().__init__()
        self.norm = nn.LayerNorm(channels)
        self.conv1 = nn.Conv2d(channels,channels,kernel_size=kernel_size,padding=kernel_size//2)
        self.t_net = nn.Linear(t_dim,channels)
        self.conv2 = nn.Conv2d(channels,channels,kernel_size=kernel_size,padding=kernel_size//2)
        self.conv2.weight.data.fill_(0)
        self.conv2.bias.data.fill_(0)
    
    def forward(self,x,t):
        xc = x.clone()
        x = self.norm(x.permute(0,2,3,1)).permute(0,3,1,2)
        x = self.conv1(x.relu())
        x = x + self.t_net(t).unsqueeze(-1).unsqueeze(-1).expand(t.shape[0],x.shape[1],x.shape[2],x.shape[3])
        x = self.conv2(x.relu())
        return x + xc

class Attention(nn.Module):

    def __init__(self,channels=128,attn_dim=32) -> None:
        super().__init__()
        self.norm = nn.LayerNorm(channels)
        self.Q = nn.Conv2d(channels,attn_dim,kernel_size=1,bias=False)
        self.K = nn.Conv2d(channels,attn_dim,kernel_size=1,bias=False)
        self.V = nn.Conv2d(channels,channels,kernel_size=1,bias=False)
        self.out_proj = nn.Conv2d(channels,channels,kernel_size=1)
        self.Q.weight.data.normal_(0,0.02)
        self.K.weight.data.normal_(0,0.02)
        self.V.weight.data.normal_(0,0.02)
    
    def forward(self,x):
        xc = x.clone()
        x = self.norm(x.permute(0,2,3,1)).permute(0,3,1,2)
        q = self.Q(x)
        k = self.K(x)
        v = self.V(x)
        attn_score = torch.einsum('bchw,bcxy->bhwxy',q,k).reshape(q.shape[0],*q.shape[-2:],-1)
        attn_score = attn_score.softmax(dim=-1).reshape(q.shape[0],*q.shape[-2:],*k.shape[-2:])
        return self.out_proj(torch.einsum('bhwxy,bcxy->bchw',attn_score,v)) + xc

class F_x_t(nn.Module):

    def __init__(self,in_channels,out_channels,out_size,kernel_size=3,t_shape=64,attn=False,residual=True) -> None:
        super().__init__()
        # self.t_channels = out_channels // 2
        # self.conv_channels = out_channels - self.t_channels
        self.t_channels = out_channels
        self.conv_channels = out_channels
        self.conv = nn.Conv2d(in_channels, self.conv_channels, kernel_size=kernel_size, padding=kernel_size//2)
        self.out_size = out_size
        self.fc = nn.Linear(t_shape, self.t_channels)
        self.attn = attn
        self.residual = residual
        if attn:
            self.attentions = nn.ModuleList([Attention(channels=self.conv_channels,attn_dim=out_channels) for _ in range(2)])
        if residual:
            self.ress = nn.ModuleList([ResidualBlock(channels=out_channels,kernel_size=kernel_size,t_dim=t_shape) for _ in range(2)])
        # self.fc = nn.Embedding(t_shape, self.t_num)

    def forward(self, x, t):
        if self.t_channels == 0:
            raise NotImplementedError()
            return self.conv(x)
        # return torch.cat([self.conv(x),self.fc(t).unsqueeze(-1).unsqueeze(-1).expand(t.shape[0], self.t_channels, self.out_size, self.out_size)],dim=1).relu()
        val = self.conv(x) + self.fc(t).unsqueeze(-1).unsqueeze(-1).expand(t.shape[0], self.t_channels, self.out_size, self.out_size)
        if self.residual:
            for i,ly in enumerate(self.ress):
                val = ly(val,t)
                if self.attn:
                    val = self.attentions[i](val)
        return val

class DDPM(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.t_embedding_dim = 32
        self.t_embedding = SinousEmbedding(dim=self.t_embedding_dim)
        self.up= nn.ModuleList([
            F_x_t(in_channels=1,out_channels=32,out_size=32,kernel_size=3,t_shape=self.t_embedding_dim),
            F_x_t(in_channels=32,out_channels=64,out_size=16,kernel_size=3,t_shape=self.t_embedding_dim,attn=True),
            F_x_t(in_channels=64,out_channels=128,out_size=8,kernel_size=3,t_shape=self.t_embedding_dim,attn=True),
            # ResidualBlock(channels=128,kernel_size=3,t_dim=self.t_embedding_dim),
            # F_x_t(in_channels=128,out_channels=128,out_size=4,kernel_size=1,t_shape=self.t_embedding_dim),
        ])
        self.middle = nn.ModuleList([
            # nn.Identity()
            F_x_t(in_channels=128,out_channels=128,out_size=4,kernel_size=3,t_shape=self.t_embedding_dim,attn=True),
            # ResidualBlock(channels=128,kernel_size=3,t_dim=self.t_embedding_dim),
            # F_x_t(in_channels=128,out_channels=128,out_size=4,kernel_size=1,t_shape=self.t_embedding_dim,attn=False),
        ])
        self.down= nn.ModuleList([
            # F_x_t(in_channels=128,out_channels=128,out_size=2,kernel_size=1,t_shape=self.t_embedding_dim),
            F_x_t(in_channels=128,out_channels=64,out_size=8,kernel_size=3,t_shape=self.t_embedding_dim,attn=True),
            F_x_t(in_channels=64,out_channels=32,out_size=16,kernel_size=3,t_shape=self.t_embedding_dim,attn=True),
            F_x_t(in_channels=32,out_channels=16,out_size=32,kernel_size=3,t_shape=self.t_embedding_dim),
        ])
        # self.end_mlp = nn.Conv2d(32,1,kernel_size=3,padding=1)
        self.end_mlp = nn.Conv2d(16,1,kernel_size=1)

    def forward(self,x,t):
        x = x.reshape(-1,1,28,28)
        x = F.pad(x,(2,2,2,2),mode='constant',value=0)
        ttensor = self.t_embedding(t) # [batch, 256]
        batch = x.shape[0]
        # xc = x.clone()            print(attn_score.shape)

        ups = []
        for i,ly in enumerate(self.up):
            x = ly(x,ttensor)
            ups.append(x.clone()) # append: 28x28, 14x14
            x = nn.AvgPool2d(2)(x)
            # print('up,',i);print_range(x)
        for i,ly in enumerate(self.middle):
            # x = ly(x,ttensor)
            x = ly(x,ttensor)
            # print('middle,',i);print_range(x)
        for i,ly in enumerate(self.down):
            x = nn.Upsample(scale_factor=2)(x) + ups.pop() # 14x14, 28x28
            x = ly(x,ttensor)
            # x = nn.Upsample(scale_factor=2)(x) + ups.pop()
            # print('down,',i);print_range(x)
        x = self.end_mlp(x)
        x = x[:,:,2:30,2:30]
        return x.reshape(batch,28*28)

# Train

In [13]:
import sys
import os

# parent_dir = os.path.abspath('/root/DeepLearning')
parent_dir = os.path.abspath('/home/zhh24/DeepLearning')

sys.path.append(parent_dir)
print('appended',parent_dir)

import utils

from tqdm import tqdm
import torch
import torch.nn.functional as F
import torchvision.utils

import matplotlib.pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.autograd.set_detect_anomaly(True)
mnist = utils.MNIST(batch_size=256,data_aug=False)
train_loader = mnist.train_dataloader
valid_loader = mnist.valid_dataloader
T=1000
# beta1=1e-4 # variance of lowest temperature
# betaT=6e-2 # variance of highest temperature
eps = 8e-3
steps=torch.linspace(0,T,steps=T+1,dtype=torch.float)
f_t=torch.cos(((steps/T+eps)/(1.0+eps))*3.14159*0.5)**2
betas=torch.clamp(1.0-f_t[1:]/f_t[:T],0.0,0.999)

# step = torch.log(torch.tensor(betaT/beta1))/(T-1)
# betas = beta1 * torch.exp(step*torch.arange(T,dtype=torch.float).to(device))
# step = (betaT-beta1)/(T-1)
# betas = torch.arange(T,dtype=torch.float,device=device) * step + beta1


alphas = 1-betas
alpha_bars = alphas.clone()
for i in range(1,T):
    alpha_bars[i] *= alpha_bars[i-1]

                    # we re-define a way to generate hyperparameters
                    # alpha_bar_0 = .9
                    # # alpha_bar_mid = .3
                    # alpha_bar_T = 1e-3
                    # alpha_bars = torch.zeros(T,dtype=torch.float)
                    # # alpha_bars[:T//2] = alpha_bar_0 + (alpha_bar_mid-alpha_bar_0) * torch.arange(T//2,dtype=torch.float,device=device) / (T//2)
                    # # alpha_bars[T//2:] = alpha_bar_mid + (alpha_bar_T-alpha_bar_mid) * torch.arange(T//2,dtype=torch.float,device=device) / (T//2)
                    # alpha_bars = alpha_bar_0 + (alpha_bar_T-alpha_bar_0) * torch.arange(T,dtype=torch.float,device=device) / T
                    # alphas = alpha_bars.clone()
                    # for i in range(1,T):
                    #     alphas[i] = alpha_bars[i] / alpha_bars[i-1]
                    # betas = 1-alphas

print(alpha_bars)
print('range of bars',alpha_bars.min(),alpha_bars.max())
# print(alphas)

sqrt = torch.sqrt
sigmas = sqrt(betas * (1-alpha_bars / alphas)/(1-alpha_bars))
sigmas[0] = 1
print('range of sigmas,',sigmas.min(),sigmas.max())
alphas = alphas.to(device)
alpha_bars = alpha_bars.to(device)
betas = betas.to(device)
sigmas = sigmas.to(device)
weights = torch.ones(T,dtype=torch.float,device=device)

@torch.no_grad()
def sample(model:DDPM,save_dir):
    x = torch.randn([100,784]).to(device)
    for t in range(T-1,-1,-1):
        sigmaz = torch.randn_like(x)*sigmas[t]
        if t==0:
            sigmaz = 0
        noise_pred = model(x,t*torch.ones(x.shape[0],dtype=torch.long,device=device))
        x0_pred = (x - noise_pred * sqrt(1 - alpha_bars[t])) / sqrt(alpha_bars[t]).clamp(-1,1)
        mean_pred = (sqrt(alphas[t]) * (1-alpha_bars[t]/alphas[t]) * x + sqrt(alpha_bars[t]/alphas[t]) * (1-alphas[t]) * x0_pred) / (1-alpha_bars[t])
        x = mean_pred + sigmaz
        # x = torch.clamp(x,0,1)
    grid = torchvision.utils.make_grid(post_process(x).reshape(-1,1,28,28).cpu(), nrow=10)
    torchvision.utils.save_image(grid, save_dir)

@torch.no_grad()
def visualize(model,save_dir):
    interval = (T-1) // 20
    x = torch.randn([10,784]).to(device)
    x_history = []
    for t in range(T-1,-1,-1):
        sigmaz = torch.randn_like(x)*((betas[t])**0.5).to(device)
        if t==0:
            sigmaz = 0
        x = (x-(1-alphas[t])/(sqrt(1-alpha_bars[t]))*model(x,t*torch.ones(x.shape[0],dtype=torch.long,device=device)))/(sqrt(alphas[t]))+sigmaz
        # x = torch.clamp(x,0,1)
        x_history.append(x)
    # print('cat.shape',torch.cat(x_history,dim=0).shape)
    grid = torchvision.utils.make_grid(post_process(torch.stack(x_history,dim=0)[::interval,...]).reshape(-1,1,28,28).cpu(), nrow=10)
    torchvision.utils.save_image(grid, save_dir)
    print('Saved visualize to',os.path.abspath(save_dir))

@torch.no_grad()
def visualize_denoise(model,save_dir):
    # get 10 images from the dataset
    x,_ = next(iter(valid_loader))
    x = x[:20,...].reshape(20,784).to(device)
    x = pre_process(x)
    t = torch.tensor([i * T // 20 for i in range(20)],dtype=torch.long,device=device)
    noise = torch.randn_like(x).reshape(-1,784)
    v1 = (sqrt(alpha_bars[t]).reshape(-1,1)*x).reshape(-1,784)
    v2 = sqrt(1-alpha_bars[t]).reshape(-1,1)*noise
    x_corr = v1+v2
    est = model(x_corr,t)
    x_rec = (x_corr - sqrt(1-alpha_bars[t]).reshape(-1,1)*est)/(sqrt(alpha_bars[t])).reshape(-1,1)
    grid_orig = torchvision.utils.make_grid(post_process(x).reshape(-1,1,28,28).cpu(), nrow=10)
    grid_corr = torchvision.utils.make_grid(post_process(x_corr).reshape(-1,1,28,28).cpu(), nrow=10)
    grid_rec = torchvision.utils.make_grid(post_process(x_rec).reshape(-1,1,28,28).cpu(), nrow=10)
    # add noise level infomation to the image
    noise_level = (1-alpha_bars[t]).reshape(-1).tolist()
    ori_mse = noise.pow(2).mean(dim=1).reshape(-1).tolist()
    mse = ((est-noise)**2).mean(dim=1).reshape(-1).tolist()
    print(noise_level)
    print(ori_mse)
    print(mse)
    grid = torch.cat([grid_orig,grid_corr,grid_rec],dim=1)
    torchvision.utils.save_image(grid, save_dir)
    print('Saved denoise to',os.path.abspath(save_dir))

def plot_loss(losses,save_dir):
    losses_vals, t_vals = zip(*losses)
    losses_vals = torch.cat(losses_vals,dim=0)
    t_vals = torch.cat(t_vals,dim=0)
    # print('t_vals',t_vals)
    # print('losses_vals',losses_vals)

    results = []
    for t in range(T):
        this_t = abs(t_vals.float()-float(t))<0.5
        results.append(torch.sum(torch.where(this_t,losses_vals,torch.tensor(0.,device=device))).item() / (torch.sum(this_t.float())+1e-3).item())
    plt.plot(results)
    plt.ylim(0,max(results)* 1.2)
    plt.savefig(save_dir)
    plt.close()
    # weights = (torch.tensor(results,device=device)) # weights
    weights = torch.ones(T,dtype=torch.float,device=device)
    # weights[:10]=0
    # weights[10:80] /= 100
    return weights

def pre_process(x):
    # do the logit transform
    # return (torch.log(x+1e-3)-torch.log(1-x+1e-3))
    return x*2-1 #MODIFIED
    return (x+1)/2

def post_process(x):
    # return torch.sigmoid(x)
    return (x+1)/2 #MODIFIED
    return x*2-1

def train(epochs,model:DDPM,optimizer,eval_interval=5):
    global weights
    for epoch in range(epochs):
        # print('weights normalized:',weights/weights.sum())
        all_ts = torch.distributions.Categorical(weights).sample((50000,))
        cnt = 0
        model.train()
        with tqdm(train_loader) as bar:
            losses = []
            for x,_ in bar:
                cnt += x.shape[0]
                x = pre_process(x.to(device))
                epss = torch.randn_like(x).reshape(-1,784).to(device)
                # ts = torch.randint(0,T,(x.shape[0],),device=device,dtype=torch.long)
                ts = all_ts[cnt-x.shape[0]:cnt]
                alpha_tbars = alpha_bars[ts]
                value = (sqrt(alpha_tbars).reshape(-1,1,1,1)*x).reshape(-1,784)+sqrt(1-alpha_tbars).reshape(-1,1)*epss
                out = model(value,ts) # [batch,784]
                # loss = ((epss-out).pow(2).mean(dim=-1) * (betas[ts])/(2*alphas[ts]*(1-alpha_tbars))).sum(dim=0)
                loss = ((epss-out).pow(2).mean(dim=-1)).mean(dim=0)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                losses.append(loss.item())
                bar.set_description('epoch {}, loss {:.4f}'.format(epoch,sum(losses)/len(losses)))

        model.eval()
        with torch.no_grad():
            with tqdm(valid_loader) as bar:
                mses = []
                losses = []
                losses_for_t = []
                for x,_ in bar:
                    x = pre_process(x.to(device))
                    epss = torch.randn_like(x).reshape(-1,784).to(device)
                    ts = torch.randint(0,T,(x.shape[0],),device=device,dtype=torch.long)
                    # print(ts)
                    alpha_tbars = alpha_bars[ts]
                    value = (sqrt(alpha_tbars).reshape(-1,1,1,1)*x).reshape(-1,784)+sqrt(1-alpha_tbars).reshape(-1,1)*epss
                    out = model(value,ts)
                    mse = F.mse_loss(epss,out)
                    mses.append(mse.item())
                    loss = ((epss-out).pow(2).mean(dim=-1))
                    # loss = (epss-out).pow(2).mean(dim=-1)
                    losses_for_t.append((loss.clone().detach(),ts))
                    loss = (loss).mean(dim=0)
                    losses.append(loss.item())
                    bar.set_description('epoch {}, MSE {:.4f}, [Valid] {:.4f}'.format(epoch,sum(mses)/len(mses),sum(losses)/len(losses)))
                    
        if epoch % eval_interval == 0:
            visualize(model,save_dir=os.path.join('./samples',f'diffuse_epoch_{epoch}.png'))
            sample(model,save_dir=os.path.join('./samples',f'sample_epoch_{epoch}.png'))
            # visualize_denoise(model,save_dir=os.path.join('./samples',f'denoise_epoch_{epoch}.png'))
            weights = plot_loss(losses_for_t,save_dir=os.path.join('./samples',f'loss_epoch_{epoch}.png'))
            torch.save(model,os.path.join('./samples',f'epoch_{epoch}.pt'))

if __name__ == '__main__':
    model = DDPM().to(device)
    print('Number parameters of the model:', sum(p.numel() for p in model.parameters()))
    print('Model strcuture:',model)
    optimizer = torch.optim.Adam(model.parameters(),lr=2e-4)
    os.makedirs('./samples',exist_ok=True)
    # sample(model,save_dir=os.path.join('./samples',f'init.png'))
    # visualize(model,save_dir=os.path.join('./samples',f'init_visualize.png'))
    train(200,model,optimizer,eval_interval=5)

appended /home/zhh24/DeepLearning
tensor([9.9996e-01, 9.9991e-01, 9.9986e-01, 9.9981e-01, 9.9975e-01, 9.9968e-01,
        9.9961e-01, 9.9953e-01, 9.9945e-01, 9.9937e-01, 9.9928e-01, 9.9918e-01,
        9.9908e-01, 9.9898e-01, 9.9887e-01, 9.9876e-01, 9.9864e-01, 9.9851e-01,
        9.9839e-01, 9.9825e-01, 9.9811e-01, 9.9797e-01, 9.9782e-01, 9.9767e-01,
        9.9751e-01, 9.9735e-01, 9.9718e-01, 9.9701e-01, 9.9683e-01, 9.9665e-01,
        9.9647e-01, 9.9627e-01, 9.9608e-01, 9.9588e-01, 9.9567e-01, 9.9546e-01,
        9.9525e-01, 9.9502e-01, 9.9480e-01, 9.9457e-01, 9.9434e-01, 9.9410e-01,
        9.9385e-01, 9.9360e-01, 9.9335e-01, 9.9309e-01, 9.9283e-01, 9.9256e-01,
        9.9229e-01, 9.9201e-01, 9.9172e-01, 9.9144e-01, 9.9115e-01, 9.9085e-01,
        9.9055e-01, 9.9024e-01, 9.8993e-01, 9.8961e-01, 9.8929e-01, 9.8897e-01,
        9.8864e-01, 9.8830e-01, 9.8796e-01, 9.8762e-01, 9.8727e-01, 9.8691e-01,
        9.8656e-01, 9.8619e-01, 9.8582e-01, 9.8545e-01, 9.8507e-01, 9.8469e-01,
      

  0%|                                                                                                                                    | 0/188 [00:00<?, ?it/s]

Number parameters of the model: 2282673
Model strcuture: DDPM(
  (t_embedding): SinousEmbedding()
  (up): ModuleList(
    (0): F_x_t(
      (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (fc): Linear(in_features=32, out_features=32, bias=True)
      (ress): ModuleList(
        (0): ResidualBlock(
          (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (t_net): Linear(in_features=32, out_features=32, bias=True)
          (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (1): ResidualBlock(
          (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (t_net): Linear(in_features=32, out_features=32, bias=True)
          (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )


epoch 0, MSE 0.1151, [Valid] 0.1151: 100%|███████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:04<00:00, 10.34it/s]


Saved visualize to /home/zhh24/samples/diffuse_epoch_0.png


...

Saved visualize to /home/zhh24/samples/diffuse_epoch_85.png


epoch 90, MSE 0.0399, [Valid] 0.0399: 100%|██████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:04<00:00, 10.29it/s]


Saved visualize to /home/zhh24/samples/diffuse_epoch_90.png


epoch 92, loss 0.0392:  22%|█████████████████████▊                                                                              | 41/188 [00:30<01:49,  1.34it/s]

: 